In [1]:
# This is code for the development of an algorithm for Isomaps and Local Linear Embedding

In [2]:
print("Hello World")

Hello World


In [3]:
# What is an Isomap? We can learn from this link: https://towardsdatascience.com/what-is-isomap-6e4c1d706b54
# or this paper: http://syllabus.cs.manchester.ac.uk/pgt/2020/COMP61021/reference/supervised-isomap.pdf

In [4]:
# What components do we need to run an Isomap function?
# The paper informs us we need 
# 1: Construct a Neighbourhood graph and set the edge lengths equal to a distance metric d(x_i,x_j)
# 2: Compute shortest graph s.t. \text{d_{G}(x_i,x_j)=d(x_i,x_j) \iff x_i,x_j are linked}
# so, the matrix of final values $D_G=\{d_G(x_i,x_j)\}$ contains the shortest path distances between all points
# 3: Construct d-dimensional embedding: Let \lambda_p be p^{th} eigenvalue of matrix \tau(DG) where \tau(D)=-\frac{HSH}{2}
# the only free parameter in this entire algortihm is actually \epsilon, oder "K" for KNN


In [5]:
# So, let's begin with the construction of a function for Isomap! Let's name it... Isomap()!
def Isomap(datamatrix,number_n_components,KNN_components):
    datamatrix = distance_mat(datamatrix, KNN_components)
    from sklearn.utils.graph import graph_shortest_path
    graph = graph_shortest_path(data, directed=False)
    graph = -0.5 * (graph ** 2)
    return mds(graph, n_components)

In [6]:
# Now, we need to create a distance matrix. This will take in a datamatrix X, and we will specify the free variable K=5
def distance_matrix(X, n_neighbors=5):
    def distance(a, b):
        return np.sqrt(sum((a - b)**2))

# The above code is easy to read, as it follows from definition of distance function in topology and Euclidian l^2 norm
# So, we need to implement this code as a full distance matrix, s.t. for all points in the dataset we have an entire matrix
# This is often a big matrix
    totaldistances = np.array([[dist(point1, point2) for point2 in X] for point1 in X])
    neighborpoints = np.zeros_like(totaldistances)
    sort_distances = np.argsort(totaldistances, axis=1)[:, 1:n_neighbors+1]
    for k,i in enumerate(sort_totaldistances):
        neighborpoints[k,i] = totaldistances[k,i]
    return neighborpoints, sort_totaldistances

In [7]:
# However, note that we must now implement the element of Multidimensional Scaling, or MDS
# MDS requires that we center our distancematrix, D
# This does not change any of the distances between the pairs of points
# However, it does allow for the resulting matrix to have useful properties! :D 
# Note, K is a matrix of shape m x m, so this tells us it is square

# Let's begin
# For full parsing of my code here, please refer to our written paper, hosted on github :) 

def CenterMatrix(K):
    number_of_samples = K.shape[0]
    Avg_for_Col = np.sum(K, axis=0)/ number_of_samples
    Avg_for_Row = (np.sum(K, axis=0)/ number_of_samples)[:, np.newaxis]
    Avg_for_entire_matrix = avg_for_Col.sum()/ number_of_samples
    #s.t.
    K -= Avg_for_Col
    K -= Avg_for_Row
    K += Avg_for_entire_matrix
    return K

In [8]:
# Now, let's focus on Linear Algebra :D my favorite topic ever
# Remember, definition for Eigenvector is given by {\displaystyle T(\mathbf {v} )=\lambda \mathbf {v} ,}
# where \lambda is a scalar in a field $F$, and is a "characteristic value" oder eigenvalue
# If V is a finite-dimensional vector space, we know that
# $Au=\lambda u$ where A is a matrix representation for T and u is the coordinate vector of v

# We now need to construct the function to compute the MDS and the corresponding eigenvectors \& eigenvalues

def MultidimensionalScaling(datamatrix, number): 
    CenterMatrix(datamatrix)
    Eigenvalue,Eigenvector = np.linalg.eig(datamatrix)
    Eigenpairs =[
        (np.abs(Eigenvalue[i]), Eigenvector[:,i]) for i in range(len(Eigenvalue))
    ]
    
# So what's the above code do? Well, we made a function which is basically PCA,
# where we're given a Matrix in the form of n x n (Hence, square matrix)
# and I specify in the number parameter, how many components I want to use 
# We than have created a Tuple of items(2), and so we pick the 2 principal components we want, by specifying 
# the eigenvectors which CONTAIN the two largest eigenvalues, thus we now need to write code to obtain the 
# transform matrix!

    Eigenpairs.sort(key=lambda x:x[0], reverse=True)
    Eigenpairs = np.array(Eigenpairs)
    W= np.hstack(
        [Eigenpairs[i,1].reshape(datamatrix.shape[1],1) for i in range(number)]
    )
    return W
# So, the code above is now creating a new rule(hence the Lambda) where we obtain the
# eigenvectors with largest eigenvalues, and obtain a matrix W s.t. W is a subspace transform
